In [1]:
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Markdown as MD
from IPython.display import display

In [2]:
# Create Connection to postgresql
# psql -h localhost -p 25432 -U musicbrainz musicbrainz_db

conn = psycopg2.connect(
    host="localhost",
    database="musicbrainz_db",
    user="musicbrainz",
    port=5432)

cursor = conn.cursor()

cursor.execute("select * from recording limit 1;")
cursor.fetchone()[0]
print("connection successful")

connection successful


## Loading Recording Tables

In [3]:
%%time
# Loading recording table SQL tables

MB_recording = pd.read_sql('SELECT gid FROM recording', con = conn, columns = ['rec-gid'])
# MB_recording.set_index('gid', inplace = True)
MB_recording.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 7.07 s, sys: 2.46 s, total: 9.53 s
Wall time: 18.6 s


,gid
0,0f42ab32-22cd-4dcf-927b-a8d9a183d68b
1,4dce8f93-45ee-4573-8558-8cd321256233
2,48fabe3f-0fbd-4145-a917-83d164d6386f
3,b30b9943-9100-4d84-9ad2-69859ea88fbb
4,b55f1db3-c6d2-4645-b908-03e1017a99c2


In [4]:
%%time
# Loading redirect table SQL table

# MB_redirects = pd.read_sql('select r.gid, rgr.gid from recording r join recording_gid_redirect rgr on rgr.new_id = r.id', con = conn)
# MB_redirects.columns = ['old', 'new']

MB_redirects = pd.read_sql('select r.gid AS new, rgr.gid AS old from recording r join recording_gid_redirect rgr on rgr.new_id = r.id', con=conn)
MB_redirects.head()

# select t.gid AS new, tgr.gid AS old from track t join track_gid_redirect tgr on tgr.new_id = t.id

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 1.24 s, sys: 150 ms, total: 1.39 s
Wall time: 10.4 s


,new,old
0,9e48544c-60ba-4d23-b741-caf7af1f4489,bffa2d3c-52f9-415a-a027-3750ec802cfa
1,deb2bafb-d22d-42e3-bf36-4a66fff9e2ff,9c3fd45f-4b0e-435f-8c85-79b7866928a8
2,7786a4a8-b7f8-4656-a1f0-887ffa6967ca,bff4f208-81de-4b69-9139-5b693d9c18b4
3,b3fad4b6-1461-4098-814f-36ba188892c3,687f1c8a-3acd-4ec0-a8ac-fb35a35796a0
4,567dd691-b596-4f29-87a0-c7eca83f9a0c,f796a23b-0fba-4af8-859d-5d480e020171


## Loading track tables

In [5]:
%%time
# Loading track table

MB_track = pd.read_sql('SELECT gid FROM track', con = conn)
MB_track.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 9.52 s, sys: 3.14 s, total: 12.7 s
Wall time: 25.1 s


,gid
0,9b02977e-a03b-4a6b-a9a9-06e722bdcd7a
1,43da7544-6283-3159-84f9-537fe823a1a7
2,0b6b6283-a5a8-4560-9fa8-f68a430d86ea
3,fa124f9a-d8ea-36a3-bed3-c817fdbe13e2
4,e56c6d3c-09cf-33a0-81c5-ceade77c35dc


In [6]:
%%time

MB_track_redirects = pd.read_sql('select t.gid AS new, tgr.gid AS old from track t join track_gid_redirect tgr on tgr.new_id = t.id', con=conn)
MB_track_redirects.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 234 ms, sys: 18.4 ms, total: 252 ms
Wall time: 925 ms


,new,old
0,5c2e31e6-8ff0-3f80-8e52-d7d6ba642aaf,d8abbf14-5945-3639-a0c9-3e9c70b1c0a4
1,742af864-28da-3fc3-9d85-ee509d83d1ce,446b27ef-92fd-3ea1-ad0a-fe1055196406
2,ec67694e-fefb-3374-9f4f-18d12233d055,403b9e19-a135-3acc-ae69-58fb0d735036
3,2a17c2f4-0e7e-39ea-acad-b88c2a503631,b61dde50-de25-3802-b706-dd0d490879ae
4,33caee67-b948-361e-bdc5-77fe0d842a88,13bee970-0129-320d-94c8-dfbc7748c692


In [7]:
%%time
# Loading canonical recording mbid table

MB_canonical = pd.read_sql('SELECT recording_mbid as old, canonical_recording_mbid as new FROM mapping.canonical_recording_redirect', con = conn)
# MB_canonical.columns = ['old', 'new']
# MB_canonical.set_index('recording-gid', inplace=True)
MB_canonical.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 2.36 s, sys: 735 ms, total: 3.09 s
Wall time: 5.99 s


,old,new
0,6ac02452-ee12-4f86-b389-bd20ba2fefcf,3e8eebfd-7613-4b3d-acbe-41709be76618
1,b4c26989-1b9e-4d50-8cde-56d6472e4bc3,3e8eebfd-7613-4b3d-acbe-41709be76618
2,601e1cf3-ad6c-4e38-9128-ba4d0d4b010f,b1050d12-b8af-409c-9cff-22759d93e240
3,35c4d840-e51f-4c07-9418-af9335b29642,f4680747-bf28-417a-ab33-af00577d8ac2
4,9ba7a9b9-a21c-4b12-8771-4c108b08b3e2,13b3875a-c89a-4be5-a6e4-0ca9164bc41d


## Loading artist tables

In [8]:
%%time
# Loading artist-mbid table for artist-conflation.

MB_artist = pd.read_sql('SELECT gid FROM artist', con = conn)
MB_artist.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 548 ms, sys: 112 ms, total: 660 ms
Wall time: 1.35 s


,gid
0,fadeb38c-833f-40bc-9d8c-a6383b38b1be
1,49add228-eac5-4de8-836c-d75cde7369c3
2,c112a400-af49-4665-8bba-741531d962a1
3,ca3f3ee1-c4a7-4bac-a16a-0b888a396c6b
4,7b4a548e-a01a-49b7-82e7-b49efeb9732c


In [9]:
%%time

MB_artist_redirects = pd.read_sql('select a.gid AS new, agr.gid AS old from artist a join artist_gid_redirect agr on agr.new_id = a.id', con=conn)
MB_artist_redirects.head()

/home/snaek/MLHD/env_mlhd/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: user 41.9 ms, sys: 1.72 ms, total: 43.7 ms
Wall time: 824 ms


,new,old
0,ad8260b2-2767-4e9b-9ece-7977fbcedadf,224efe8c-c070-4e8a-9402-d41d52f5b4f1
1,c8f70fe2-a3fb-48cd-af02-ce5df49a7875,549de92e-a454-47f3-a26f-900985cf4431
2,21074eb7-3849-4d0e-bd4d-5447ff175c45,aca98fdf-f1b5-4831-8d58-ab2541c63863
3,21074eb7-3849-4d0e-bd4d-5447ff175c45,d442c9b0-2052-4fda-ad7b-e254d70cd61e
4,21074eb7-3849-4d0e-bd4d-5447ff175c45,33da8f6e-4c9c-4a54-903a-d10788ceea77


# Loading Data

In [10]:
# Reads a list of file paths and reads + compiles data into a single pd.DataFrame
def read_files(file_path_repo):
    
    # init new empty main dataframe
    df = pd.DataFrame(columns = ['timestamp', 'artist-MBID', 'release-MBID', 'recording-MBID'])
    
    # Open a file with MLHD file paths to process
    with open(file_path_repo, 'r') as f:
        file_paths = f.readlines()
        file_paths= [item.strip() for item in file_paths]
    
    # Read files and compile into single df
    for pth in file_paths:
        temp = pd.read_csv(pth, sep='\t', names=['timestamp', 'artist-MBID', 'release-MBID', 'recording-MBID'])
        temp = temp[-temp['recording-MBID'].isna()]

        df = pd.concat([df, temp])
    
    return df

df = read_files('random_file_paths.txt')
print(df.shape)
df.head()

(3661057, 4)


,timestamp,artist-MBID,release-MBID,recording-MBID
0,1108412731,f4a31f0a-51dd-4fa7-986d-3095c40c5ed9,426c5c82-4472-4f7d-b1d9-9f928d338340,1deb956c-5439-4fbb-b026-5adb4330a934
1,1108422818,db999c3f-f243-4a5f-88d6-0c25243b6661,6079df6b-2c00-4fd6-b015-0e303eedf4fd,14e9eb4e-155d-46ff-9a83-a8d5e1936c81
2,1108423325,ce58d854-7430-4231-aa44-97f0144b3372,e427c52c-60f4-3df4-9493-2df0734d85aa,eced9a9b-cd59-40f8-a580-f27094bd8a89
3,1108428897,ce58d854-7430-4231-aa44-97f0144b3372,e427c52c-60f4-3df4-9493-2df0734d85aa,eced9a9b-cd59-40f8-a580-f27094bd8a89
4,1108429140,ce58d854-7430-4231-aa44-97f0144b3372,e427c52c-60f4-3df4-9493-2df0734d85aa,eced9a9b-cd59-40f8-a580-f27094bd8a89


In [11]:
null_count_artist = df['artist-MBID'].isnull().value_counts()
null_count_rec = df['recording-MBID'].isnull().value_counts()
null_count_rel = df['release-MBID'].isnull().value_counts()

def get_null_stats(val_count, attr_name):
    print("Number of NOT-null rows in {} = {}".format(attr_name, (val_count[0]/val_count.sum())*100))

get_null_stats(null_count_artist, 'artist-MBID')
get_null_stats(null_count_rec, 'recording-MBID')
get_null_stats(null_count_rel, 'release-MBID')

Number of NOT-null rows in artist-MBID = 98.87100911021052
Number of NOT-null rows in recording-MBID = 100.0
Number of NOT-null rows in release-MBID = 75.44897552810568


## Architechture:

1. Take a chunk of MBIDs (Test optimal chunk sizes too. Current optimal > 253k rows)
2. "Squish" series
    - i.e. Only take unique values from the series.
3. Pass squished series thorugh the following:
    - Get rec-mbids, and check if they exist in the track table.
    - Get rec-mbids, and check if they exist in the track_gid_redirect table.
    - Get rec-mbids, and check if they exist in the recording table.
    - Get rec-mbids that don't exist in recording, and pass it through MB_redirect
    - Get artist-mbids, and check if they exist in the artist table.
    - Get artist-mbids, and check if they exist in the artist_gid_redirect table.

4. "Unsquish" the series.
    - i.e. Take processed output for squished values, and apply them to unsquished values.
    - This process ensures processing only on unique values.
    The output for this processing is then applied to duplicate values as well.

In [12]:
# '''Squish function: 
# 1. Takes in input series with index number and recording-MBID.
# 2. Makes a mapping table with recording-MBIDs as the index, 
# and a series of row-indices with that MBID as the values.'''

# # def squish(input_series):

# '''
# 1. take inp_series
# 2. Generate empty mapping_df where:
#     - index = MBID
#     - value = series of indices from inp_series
# 2. start traversing
# 3. if new ID: 
#     - Add ID to mapping_df.index
#     - Set the value in mapping_df as a list of newly updated indices for inp_series
# 4. if not new ID:
#     - Add ID to list of indices for inp_series.
#     - Update this ID in mapping_df.index
# '''

# # inp = df['recording-MBID'].reset_index(drop=True)
# # inp_unique = inp.unique()

# # mapping_df = pd.DataFrame(index=inp_unique)
# # mapping_df

# # Tackle this later. Focus on basic unoptimized code first!

In [13]:
# A generic function for queries a series into another series.
    # Returns a series of boolean values corresponding to series_of_mbids 
    # (bool specifies if value exists in recording table or not.)

def query_in(series_to_query, series_to_query_in):
    # Queries all mbids in the recording table. Returns mbids that are present in recording table.
    mbids_in_series = series_to_query_in[series_to_query_in.isin(series_to_query)]
    
    # Makes a boolean map for all mbids in series_of_mbids
    bool_map = series_to_query.isin(mbids_in_series)
    
    return bool_map

## Testing Track MBIDs

In [14]:
%%time

# Just to prove that absolutely none of the of the 381k unique mbids are track-mbids disguised as rec-mbid.
"""
Input: all unique recording-MBIDs from the input dataset.
Output: Boolmap of all recording-MBIDs that belong to the MB "track" table.
"""

in_for_track = pd.Series(df['recording-MBID'].unique())
out_for_track = query_in(in_for_track, MB_track)

out_for_track.value_counts()

CPU times: user 2.7 s, sys: 275 ms, total: 2.98 s
Wall time: 2.98 s


False    381145
dtype: int64

In [15]:
%%time

"""
Input: all unique recording-MBIDs from the input dataset.
Output: Boolmap of all recording-MBIDs that belong to the MB "track_gid_redirect" table.
"""

in_for_track = pd.Series(df['recording-MBID'].unique())
out_for_track_redir = query_in(in_for_track, MB_track_redirects.old)

track_mbid_in_r_mbid = out_for_track.value_counts()
track_mbid_in_r_mbid

CPU times: user 565 ms, sys: 0 ns, total: 565 ms
Wall time: 563 ms


False    381145
dtype: int64

## Testing for recording-MBID

In [16]:
%%time
"""
Input: all unique recording-MBIDs from the input dataset.
Output: Boolmap of all recording-MBIDs that belong to the entity table (MB "recording" table.)
"""

in_for_rec = pd.Series(df['recording-MBID'].unique())
out_for_rec = query_in(in_for_rec, MB_recording.gid)

r_mbid_in_r_mbid = out_for_rec.value_counts()
r_mbid_in_r_mbid

CPU times: user 8.95 s, sys: 0 ns, total: 8.95 s
Wall time: 8.95 s


True     295214
False     85931
dtype: int64

## Testing for rec-MBID redirects

In [17]:
%%time

# Testing if MBIDs that DONT exist in MB recording table have any redirects
"""
Input: all unique recording-MBIDs from the input dataset that DON'T belong the MB "recording" table.
Output: Boolmap of all recording-MBIDs that have a redirect available.
"""
in_for_rec_redir = in_for_rec[-out_for_rec]

out_for_rec_redir = query_in(in_for_rec_redir, MB_redirects.old)

out_for_rec_redir.value_counts()

CPU times: user 716 ms, sys: 0 ns, total: 716 ms
Wall time: 713 ms


True     84783
False     1148
dtype: int64

In [18]:
in_for_rec_redir[-out_for_rec_redir].to_csv('unk_ids/unk_rec_mbids.txt')

In [19]:
%%time

# Testing if MBIDs that exist in MB recording table have any redirects
"""
Input: all unique recording-MBIDs from the input dataset that belong the MB "recording" table.
Output: Boolmap of all recording-MBIDs that have a redirect available.
"""

in_for_rec_redir_2 = in_for_rec[out_for_rec]

out_for_rec_redir_2 = query_in(in_for_rec_redir_2, MB_redirects.old)

out_for_rec_redir_2.value_counts()

CPU times: user 628 ms, sys: 0 ns, total: 628 ms
Wall time: 623 ms


False    295214
dtype: int64

In [20]:
# # Fetching redirected MBIDs.

# # in_for_redir = pd.Series(df['recording-MBID'].unique())
# in_for_redir = in_for_rec[out_for_rec]
# out_for_redir = MB_redirects.old.isin(in_for_redir)

# out_for_redir.value_counts()

## Checking if artist-MBIDs belong to artist ID.

In [21]:
# Checking if artist-MBIDs belong to artist ID.
"""
Input: all unique artist-MBIDs from the dataset.
Output: Boolmap of all artist-MBIDs that are present in the MB "artist" table.
"""

input_artists = pd.Series(df['artist-MBID'].dropna().unique())

output_artists = query_in(input_artists, MB_artist.gid)
output_artists.value_counts()

True     30839
False     2596
dtype: int64

In [44]:
# Checking if artist-MBIDs belong to artist ID.
"""
Input: all unique artist-MBIDs from the dataset that DON'T belong to the MB "artist" table.
Output: Boolmap of all artist-MBIDs that are NOT present in the MB "artist" table and have a redirect available.
"""

input_artists_redirect = input_artists[-output_artists]

output_artists_redirect = query_in(input_artists_redirect, MB_artist_redirects.old)
output_artists_redirect.value_counts()

True     2453
False     143
dtype: int64

In [23]:
input_artists_redirect[-output_artists_redirect].to_csv('unk_ids/unk_artist_mbids.txt')

In [45]:
# Checking if artist-MBIDs belong to artist ID.
"""
Input: all unique artist-MBIDs from the dataset that belong to the MB "artist" table.
Output: Boolmap of all artist-MBIDs that are present in the MB "artist" table and have a redirect available.
"""

input_artists_redirect_2 = input_artists[output_artists]

output_artists_redirect_2 = query_in(input_artists_redirect_2, MB_artist_redirects.old)
output_artists_redirect_2.value_counts()

False    30839
dtype: int64

In [30]:
report_track = """
# Analysis Report

## Track-MBIDs

### Number of rec-MBIDs in track:
    - Total Input: {}
    - False: {}

### Number of rec-MBIDs in track_gid_redirect:
    - Total Input: {}
    - False: {}
""".format(len(in_for_track),
            out_for_track.value_counts()[0],
            len(in_for_track),
            out_for_track_redir.value_counts()[0])


display(MD(report_track))


# Analysis Report

## Track-MBIDs

### Number of rec-MBIDs in track:
    - Total Input: 381145
    - False: 381145

### Number of rec-MBIDs in track_gid_redirect:
    - Total Input: 381145
    - False: 381145


In [50]:
report_rec = """
## recording-MBIDs
### Number of rec-MBIDs in recording-MBID:
    - Total Input: {} rows
    - (True, False) = {}, {}
    - i.e. {}% rec-MBIDs belong to the recording-MBID table.


### Number of UNKNOWN rec-MBIDs in redirected-MBID:
    - Note: "UNKNOWN rec-MBIDs" = rec-MBIDs from the MLHD that were NOT found in the MB recording table.
    - Total Input: {} rows
    - (True, False): {}, {}
    - i.e. {}% of UNKNOWN rec-MBIDs have corresponding redirects.


### Number of rec-MBIDs in canonical-MBID table:
    - _pending_
""".format(len(in_for_rec),
            out_for_rec.value_counts()[1],
            out_for_rec.value_counts()[0],
            str((out_for_rec.value_counts()[1]/len(in_for_rec))*100)[:5],
            
            len(in_for_rec_redir),
            out_for_rec_redir.value_counts()[1],
            out_for_rec_redir.value_counts()[0],
            str((out_for_rec_redir.value_counts()[1]/len(in_for_rec_redir))*100)[:5],
            )

display(MD(report_rec))


## recording-MBIDs
### Number of rec-MBIDs in recording-MBID:
    - Total Input: 381145 rows
    - (True, False) = 295214, 85931
    - i.e. 77.45% rec-MBIDs belong to the recording-MBID table.


### Number of UNKNOWN rec-MBIDs in redirected-MBID:
    - Note: "UNKNOWN rec-MBIDs" = rec-MBIDs from the MLHD that were NOT found in the MB recording table.
    - Total Input: 85931 rows
    - (True, False): 84783, 1148
    - i.e. 98.66% of UNKNOWN rec-MBIDs have corresponding redirects.


### Number of rec-MBIDs in canonical-MBID table:
    - _pending_


In [46]:
report_artist = """
### Number of artist-MBIDs in MB artist table:
    - Total Input: {}
    - True, False = {}, {}
    - i.e. {}% artist-MBIDs exist in the MB artist table.

### Number of UNKNOWN artist-MBIDs in MB artist_gid_redirect table:
    - Total Input: {}
    - True, False = {}, {}
    - i.e. {}% artist-MBIDs exist in the MB artist table.
""".format(len(input_artists),
            output_artists.value_counts()[0],
            output_artists.value_counts()[1],
            str((output_artists.value_counts()[1]/len(input_artists))*100)[:5],
            len(input_artists_redirect),
            output_artists_redirect.value_counts()[0],
            output_artists_redirect.value_counts()[1],
            str((output_artists_redirect.value_counts()[1]/len(input_artists_redirect))*100)[:5]
            )

display(MD(report_artist))


### Number of artist-MBIDs in MB artist table:
    - Total Input: 33435
    - True, False = 2596, 30839
    - i.e. 92.23% artist-MBIDs exist in the MB artist table.

### Number of UNKNOWN artist-MBIDs in MB artist_gid_redirect table:
    - Total Input: 2596
    - True, False = 143, 2453
    - i.e. 94.49% artist-MBIDs exist in the MB artist table.
